In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/abccsvv/abc.csv


In [2]:
from ecies.utils import generate_eth_key, generate_key
from ecies import encrypt, decrypt
eth_k = generate_eth_key()
sk_hex = eth_k.to_hex()  # hex string
pk_hex = eth_k.public_key.to_hex()  # hex string
data = b'this is a test'
print(encrypt(pk_hex, data))
print("dec", decrypt(sk_hex, encrypt(pk_hex, data)))

secp_k = generate_key()
sk_bytes = secp_k.secret  # bytes
pk_bytes = secp_k.public_key.format(True)  # bytes
print(encrypt(pk_bytes, data))
decrypt(sk_bytes, encrypt(pk_bytes, data))


ModuleNotFoundError: No module named 'ecies'

In [3]:
from Cryptodome.Cipher import AES
import scrypt, os, binascii

def encrypt_AES_GCM(msg, password, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce=b'0123456789abcdef')
    ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
    return (kdfSalt, ciphertext, aesCipher.nonce, authTag)

def decrypt_AES_GCM(encryptedMsg, password):
    (kdfSalt, ciphertext, nonce, authTag) = encryptedMsg
    secretKey = scrypt.hash(password, kdfSalt, N=16384, r=8, p=1, buflen=32)
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

def e(msg, secretKey):
    password = b's3kr'
    encryptedMsg = encrypt_AES_GCM(msg, password, secretKey)
    print("encryptedMsg", {
        'kdfSalt': binascii.hexlify(encryptedMsg[0]),
        'ciphertext': binascii.hexlify(encryptedMsg[1]),
        'aesIV': binascii.hexlify(encryptedMsg[2]),
        'authTag': binascii.hexlify(encryptedMsg[3])
    })

    decryptedMsg = decrypt_AES_GCM(encryptedMsg, password)
    print("decryptedMsg", decryptedMsg)

kdfSalt = get_random_bytes(AES.block_size)
password = b's3krfjgkh'
secretKey = scrypt.hash(password, kdfSalt, N=16384, r=8, p=1, buflen=32)
print(binascii.hexlify(secretKey))
msg = b'A'
e(msg, secretKey)
msg = b'B'
e(msg, secretKey)

ModuleNotFoundError: No module named 'Cryptodome'

In [4]:
from base64 import b64encode, b64decode
import hashlib
from Cryptodome.Cipher import DES3
import os
from Cryptodome.Random import get_random_bytes
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time



for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv('../input/abccsvv/abc.csv')

#df.to_csv(r'./abcghbh.csv', index = False)


def encrypt(plain_text, password):
    # generate a random salt
    #salt = get_random_bytes(AES.block_size)
    salt = get_random_bytes(8)
    key = b'-8B key-'
    key = DES3.adjust_key_parity(get_random_bytes(24))
    

    # use the Scrypt KDF to get a private key from the password
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create cipher config
    #cipher_config = AES.new(private_key, AES.MODE_GCM)
    cipher_config = DES3.new(key, DES.MODE_OFB)


    # return a dictionary with the encrypted text
    #cipher_text, tag = cipher_config.encrypt_and_digest(bytes(plain_text, 'utf-8'))
    cipher = cipher_config.encrypt(b'plain_text')
    return cipher
    #return {
    #    'cipher_text': b64encode(cipher_text).decode('utf-8'),
    #    'salt': b64encode(salt).decode('utf-8'),
     #   'nonce': b64encode(cipher_config.nonce).decode('utf-8'),
    #    'tag': b64encode(tag).decode('utf-8')
    #}


def decrypt(enc_dict, password):
    # decode the dictionary entries from base64
    salt = b64decode(enc_dict['salt'])
    cipher_text = b64decode(enc_dict['cipher_text'])
    nonce = b64decode(enc_dict['nonce'])
    tag = b64decode(enc_dict['tag'])
    

    # generate the private key from the password and salt
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create the cipher config
    #cipher = AES.new(private_key, AES.MODE_GCM, nonce=nonce)
    cipher = DES3.new(private_key, DES.MODE_OFB)

    # decrypt the cipher text
    decrypted = cipher.decrypt_and_verify(cipher_text, tag)

    return decrypted

convert_dict = {'id': object} 
df = df.astype(convert_dict) 

def main():
    password = "KAWIN"
    
    for i in range(10):
        encrypted = encrypt(str(df.at[i+1, 'id']), password)
        #df.at[i+1, 'id'] = encrypted['cipher_text']
        df.at[i+1, 'id'] = encrypted


    print("--- %s seconds ---" % (time.time() - start_time))    
    # First let us encrypt secret message
   
    #print(encrypted)

    # Let us decrypt using our original password
    #decrypted = decrypt(encrypted, password)
    #print(bytes.decode(decrypted))

start_time = time.time()
main()

#pip install pycryptodome

ModuleNotFoundError: No module named 'Cryptodome'

In [5]:
from base64 import b64encode, b64decode
import hashlib
from Cryptodome.Cipher import AES
import os
from Cryptodome.Random import get_random_bytes
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time



for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv('../input/abccsvv/abc.csv')

#df.to_csv(r'./abcghbh.csv', index = False)


def encrypt(plain_text, password):
    # generate a random salt
    salt = get_random_bytes(AES.block_size)

    # use the Scrypt KDF to get a private key from the password
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create cipher config
    cipher_config = AES.new(private_key, AES.MODE_GCM)

    # return a dictionary with the encrypted text
    cipher_text, tag = cipher_config.encrypt_and_digest(bytes(plain_text, 'utf-8'))
    return {
        'cipher_text': b64encode(cipher_text).decode('utf-8'),
        'salt': b64encode(salt).decode('utf-8'),
        'nonce': b64encode(cipher_config.nonce).decode('utf-8'),
        'tag': b64encode(tag).decode('utf-8')
    }


def decrypt(enc_dict, password):
    # decode the dictionary entries from base64
    salt = b64decode(enc_dict['salt'])
    cipher_text = b64decode(enc_dict['cipher_text'])
    nonce = b64decode(enc_dict['nonce'])
    tag = b64decode(enc_dict['tag'])
    

    # generate the private key from the password and salt
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create the cipher config
    cipher = AES.new(private_key, AES.MODE_GCM, nonce=nonce)

    # decrypt the cipher text
    decrypted = cipher.decrypt_and_verify(cipher_text, tag)

    return decrypted

convert_dict = {'id': object} 
df = df.astype(convert_dict) 

def main():
    password = "KAWIN"
    
    for i in range(len(df)-1):
        encrypted = encrypt(str(df.at[i+1, 'id']), password)
        df.at[i+1, 'id'] = encrypted['cipher_text']


    print("--- %s seconds ---" % (time.time() - start_time))    
    # First let us encrypt secret message
   
    #print(encrypted)

    # Let us decrypt using our original password
    decrypted = decrypt(encrypted, password)
    print(bytes.decode(decrypted))

start_time = time.time()
main()

#pip install pycryptodome x 

ModuleNotFoundError: No module named 'Cryptodome'